In [ ]:
import os
#'OS' permite manipular pastas, arquivos, executar comandos do sistema e gerenciar processos
import pandas as pd
import numpy as np
from google.colab import drive
import time

#para construção de modelo tensorflow
#o tensorflow para criar , treinar e implantar modelos de aprendizado de maquina
import tensorflow as tf
from tensorflow import keras
#keras para a criação de modelos de aprendizado profundo, tarefas como classificação de imagens, detecção de objetos e regressão.
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
#TfidfVectorizer para converter os textos em vetores numéricos.
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import requests
from io import BytesIO
#pillow ou opencv para manipulação de imgs
from PIL import Image



## Etapas para treinamento da RN
1.   Carregamento e preparação dos dados: Importar o dataset e dividir em conjuntos de treino e teste.
2.  Construção do modelo: Criar uma rede neural sequencial com camadas densas.
3.  Compilação do modelo: Definir a função de perda, otimizador e métricas de avaliação.
4.  Treinamento do modelo: Ajustar o modelo aos dados de treino.
5.  Avaliação do modelo: Testar a precisão do modelo com os dados de teste.
6.  Fazer previsões: Usar o modelo treinado para classificar novas imagens.


#1.Carregamento e Preparação dos dados

##a - montar o meu google drive



In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##b - Definir o caminho do diretório onde suas imagens estão armazenadas no Google Drive


In [ ]:
image_dir = '/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/'
image_dir

'/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/'

##c - Carregar o meu CSV

In [ ]:
df_Arvores = pd.read_csv('/content/drive/My Drive/Colab Notebooks/catalogo.csv',encoding='utf-8' , sep=';')
#remover linhas vazias
df_Arvores = df_Arvores.dropna()
df_Arvores

,nome,url,descrição,target
0,tree_01,Dataset_Arvores\tree_01.jpg,arvore saudavel de pe,1
1,tree_02,Dataset_Arvores\tree_02.jpg,arvore saudavel de pe invertida,1
2,tree_03,Dataset_Arvores\tree_03.jpg,arvore saudavel de pe,1
3,tree_04,Dataset_Arvores\tree_04.jpg,arvore saudavel de pe,1
4,tree_05,Dataset_Arvores\tree_05.jpg,arvore saudavel de pe,1
...,...,...,...,...
75,tree_76,Dataset_Arvores\tree_76.jpg,arvore em pe com galhos inclinados invertida,3
76,tree_77,Dataset_Arvores\tree_77.jpg,arvore em pe com galhos inclinados invertida,3
77,tree_78,Dataset_Arvores\tree_78.jpg,arvore em pe com galhos inclinados,3
78,tree_79,Dataset_Arvores\tree_79.jpg,arvore em pe com galhos inclinados,3


##d - Ajustar o caminho das imagens no DataFrame

In [ ]:

#df_Arvores['url'] = df_Arvores['url'].apply(lambda x: x.replace('\\', '/'))

##e - Adicionar o caminho completo das imagens

In [ ]:
df_Arvores['image_path'] = df_Arvores['url'].apply(lambda x: os.path.join(image_dir, os.path.basename(x)))

##f - Remover colunas desnecessárias

In [ ]:
#df_Arvores = df_Arvores.drop(columns=['Unnamed: 4'])

##g - Verificar o DataFrame ajustado

In [ ]:
print(df_Arvores.head())
print(df_Arvores.columns)

      nome                          url                        descrição  \
0  tree_01  Dataset_Arvores\tree_01.jpg            arvore saudavel de pe   
1  tree_02  Dataset_Arvores\tree_02.jpg  arvore saudavel de pe invertida   
2  tree_03  Dataset_Arvores\tree_03.jpg            arvore saudavel de pe   
3  tree_04  Dataset_Arvores\tree_04.jpg            arvore saudavel de pe   
4  tree_05  Dataset_Arvores\tree_05.jpg            arvore saudavel de pe   

   target                                         image_path  
0       1  /content/drive/My Drive/Colab Notebooks/Datase...  
1       1  /content/drive/My Drive/Colab Notebooks/Datase...  
2       1  /content/drive/My Drive/Colab Notebooks/Datase...  
3       1  /content/drive/My Drive/Colab Notebooks/Datase...  
4       1  /content/drive/My Drive/Colab Notebooks/Datase...  
Index(['nome', 'url', 'descrição', 'target', 'image_path'], dtype='object')


##h - Dividir as características e os valores de target


In [ ]:
caracteristica = df_Arvores[['nome', 'url', 'descrição']].values
valoracao = df_Arvores['target'].values

##i - Dividir os dados em conjuntos de treino e teste


In [ ]:
train_caracteristica, test_caracteristica, train_valoracao, test_valoracao = train_test_split(caracteristica, valoracao, test_size=0.2, random_state=42)


In [ ]:
# Concatenando texto das características para o modelo de texto
train_text = train_caracteristica[:, 0] + ' ' + train_caracteristica[:, 1] + ' ' + train_caracteristica[:, 2]
test_text = test_caracteristica[:, 0] + ' ' + test_caracteristica[:, 1] + ' ' + test_caracteristica[:, 2]


#2.Construção do Modelo

##a - Passo 2: Construir o pipeline de vetorização e modelo

In [ ]:
pipeline = make_pipeline(TfidfVectorizer(), RandomForestClassifier(n_estimators=100, random_state=42))

##b - Passo 3: Treinar o modelo

In [ ]:
pipeline.fit(train_text, train_valoracao)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

##c - Passo 4: Fazer previsões


In [ ]:
predictions = pipeline.predict(test_text)

##d - Avaliar o modelo

In [ ]:
accuracy = accuracy_score(test_valoracao, predictions)
report = classification_report(test_valoracao, predictions)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(f'Accuracy: {accuracy}')
print(f'Relatório de Classificação - Reconhecimento de imagem:\n{report}')

Accuracy: 0.625
Relatório de Classificação - Reconhecimento de imagem:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.62      1.00      0.77        10
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         2

    accuracy                           0.62        16
   macro avg       0.16      0.25      0.19        16
weighted avg       0.39      0.62      0.48        16



In [ ]:
# Verificar o DataFrame ajustado
print(df_Arvores.head())
print(df_Arvores.columns)

      nome                          url                        descrição  \
0  tree_01  Dataset_Arvores\tree_01.jpg            arvore saudavel de pe   
1  tree_02  Dataset_Arvores\tree_02.jpg  arvore saudavel de pe invertida   
2  tree_03  Dataset_Arvores\tree_03.jpg            arvore saudavel de pe   
3  tree_04  Dataset_Arvores\tree_04.jpg            arvore saudavel de pe   
4  tree_05  Dataset_Arvores\tree_05.jpg            arvore saudavel de pe   

   target                                         image_path  
0       1  /content/drive/My Drive/Colab Notebooks/Datase...  
1       1  /content/drive/My Drive/Colab Notebooks/Datase...  
2       1  /content/drive/My Drive/Colab Notebooks/Datase...  
3       1  /content/drive/My Drive/Colab Notebooks/Datase...  
4       1  /content/drive/My Drive/Colab Notebooks/Datase...  
Index(['nome', 'url', 'descrição', 'target', 'image_path'], dtype='object')



##e - Função para carregar uma imagem a partir de um caminho

In [ ]:

#def load_image_from_path(image_path):
 #   img = load_img(image_path, target_size=(224, 224))  # Ajuste o tamanho da imagem conforme necessário
  #  img_array = img_to_array(img)
   # img_array = np.expand_dims(img_array, axis=0)
   # return img_array

###Essa abordagem é especialmente útil para identificar a possibilidade de queda de árvores com base nas informações textuais disponíveis, como nome, URL e descrição.

#3.Compilação do Modelo

##a - Função para carregar uma imagem a partir de um caminho


In [ ]:
def load_image_from_path(image_path):
    img = image.load_img(image_path, target_size=(224, 224))  # Ajuste o tamanho da imagem conforme necessário
    img_array = image.img_to_array(img)
    return img_array

##b - Verificar se a coluna 'image_path' foi criada corretamente

In [ ]:
# Verificar se os caminhos das imagens existem
df_Arvores['image_exists'] = df_Arvores['image_path'].apply(lambda x: os.path.exists(x))
print(df_Arvores[~df_Arvores['image_exists']])  # Exibe as linhas onde o caminho da imagem não existe


#print(df_Arvores[['image_path']].head())
df_Arvores



       nome                          url  \
0   tree_01  Dataset_Arvores\tree_01.jpg   
1   tree_02  Dataset_Arvores\tree_02.jpg   
2   tree_03  Dataset_Arvores\tree_03.jpg   
3   tree_04  Dataset_Arvores\tree_04.jpg   
4   tree_05  Dataset_Arvores\tree_05.jpg   
..      ...                          ...   
75  tree_76  Dataset_Arvores\tree_76.jpg   
76  tree_77  Dataset_Arvores\tree_77.jpg   
77  tree_78  Dataset_Arvores\tree_78.jpg   
78  tree_79  Dataset_Arvores\tree_79.jpg   
79  tree_80  Dataset_Arvores\tree_80.jpg   

                                       descrição  target  \
0                          arvore saudavel de pe       1   
1                arvore saudavel de pe invertida       1   
2                          arvore saudavel de pe       1   
3                          arvore saudavel de pe       1   
4                          arvore saudavel de pe       1   
..                                           ...     ...   
75  arvore em pe com galhos inclinados invertida   

,nome,url,descrição,target,image_path,image_exists
0,tree_01,Dataset_Arvores\tree_01.jpg,arvore saudavel de pe,1,/content/drive/My Drive/Colab Notebooks/Datase...,False
1,tree_02,Dataset_Arvores\tree_02.jpg,arvore saudavel de pe invertida,1,/content/drive/My Drive/Colab Notebooks/Datase...,False
2,tree_03,Dataset_Arvores\tree_03.jpg,arvore saudavel de pe,1,/content/drive/My Drive/Colab Notebooks/Datase...,False
3,tree_04,Dataset_Arvores\tree_04.jpg,arvore saudavel de pe,1,/content/drive/My Drive/Colab Notebooks/Datase...,False
4,tree_05,Dataset_Arvores\tree_05.jpg,arvore saudavel de pe,1,/content/drive/My Drive/Colab Notebooks/Datase...,False
...,...,...,...,...,...,...
75,tree_76,Dataset_Arvores\tree_76.jpg,arvore em pe com galhos inclinados invertida,3,/content/drive/My Drive/Colab Notebooks/Datase...,False
76,tree_77,Dataset_Arvores\tree_77.jpg,arvore em pe com galhos inclinados invertida,3,/content/drive/My Drive/Colab Notebooks/Datase...,False
77,tree_78,Dataset_Arvores\tree_78.jpg,arvore em pe com galhos inclinados,3,/content/drive/My Drive/Colab Notebooks/Datase...,False
78,tree_79,Dataset_Arvores\tree_79.jpg,arvore em pe com galhos inclinados,3,/content/drive/My Drive/Colab Notebooks/Datase...,False


In [ ]:
import os

# Verificar todos os arquivos no diretório de imagens
image_dir = '/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/'
arquivos_imagens = os.listdir(image_dir)
print(arquivos_imagens)  # Lista de todos os arquivos no diretório de imagens


['tree_10.jpg', 'tree_07.jpg', 'tree_06.jpg', 'tree_04.jpg', 'tree_02.jpg', 'tree_08.jpg', 'tree_09.jpg', 'tree_01.jpg', 'tree_03.jpg', 'tree_05.jpg', 'tree_120.jpg', 'tree_125.jpg', 'tree_122.jpg', 'tree_121.jpg', 'tree_119.jpg', 'tree_123.jpg', 'tree_12.jpg', 'tree_118.jpg', 'tree_111.jpg', 'tree_117.jpg', 'tree_110.jpg', 'tree_114.jpg', 'tree_112.jpg', 'tree_115.jpg', 'tree_11.jpg', 'tree_113.jpg', 'tree_126.jpg', 'tree_124.jpg', 'tree_109.jpg', 'tree_102.jpg', 'tree_104.jpg', 'tree_100.jpg', 'tree_103.jpg', 'tree_105.jpg', 'tree_106.jpg', 'tree_101.jpg', 'tree_108.jpg', 'tree_107.jpg', 'tree_116.jpg', 'tree_140.jpg', 'tree_141.jpg', 'tree_139.jpg', 'tree_143.jpg', 'tree_137.jpg', 'tree_14.jpg', 'tree_138.jpg', 'tree_147.jpg', 'tree_15.jpg', 'tree_149.jpg', 'tree_148.jpg', 'tree_151.jpg', 'tree_150.jpg', 'tree_146.jpg', 'tree_153.jpg', 'tree_144.jpg', 'tree_145.jpg', 'tree_130.jpg', 'tree_129.jpg', 'tree_132.jpg', 'tree_128.jpg', 'tree_131.jpg', 'tree_133.jpg', 'tree_135.jpg', 'tree

##c - Carregar as imagens


In [ ]:
df_Arvores['image'] = df_Arvores['image_path'].apply(load_image_from_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/Dataset_Arvores\\tree_01.jpg'

#4 - Treinamento do Modelo

##a - Obter as imagens e os rótulos

In [ ]:
imagens = np.array(df_Arvores['image'].tolist())
rotulos = df_Arvores['target'].values

##b - Verificar as formas das imagens


In [ ]:
print(imagens.shape)

##c - Dividir as imagens e rótulos em conjuntos de treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imagens, rotulos, test_size=0.2, random_state=42)

##d - Construir o modelo CNN

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # Supondo que você tenha 10 classes
])


##e - Compilar o modelo

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
##g - Treinar o modelo, lembrando que o modelo espera o seguinte formato de entrada (None, 224, 224, 3):

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


##h - Avaliar o modelo

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')


##i - Plotar a acurácia e a perda durante o treinamento

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()


In [ ]:
df_Arvores.head()

In [ ]:
#montar um dataframe com o resultado do modelo das arvores que possuem target 3, 4, 5 com "Latitude", "Longitude" e "prob%"
#df_resultado = df_Arvores[['Latitude', 'Longitude', 'prob%', 'Label']]
df_resultado = df_Arvores[['Latitude', 'Longitude']]

# 5 - Avaliação do Modelo - aqui posso usar várias métricas para melhor avaliação

##a - Avaliação com Dados de Teste: Importante para obter a acurácia geral e a perda no conjunto de teste.

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

##b - Previsões do conjunto de testes

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

##c - Matriz de Confusão: Ajuda a visualizar os tipos de erros de classificação

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Classe Prevista')
plt.ylabel('Classe Verdadeira')
plt.title('Matriz de Confusão')
plt.show()

##d - Relatório de Classificação


In [ ]:
class_report = classification_report(y_test, y_pred_classes)
print('Classification Report:')
print(class_report)

##e - Visualizar alguns erros de classificação


In [ ]:
incorrect = np.where(y_pred_classes != y_test)[0]
print(f'Number of misclassified samples: {len(incorrect)}')

In [ ]:
##f - Mostrar algumas imagens classificadas incorretamente

In [ ]:

num_show = 5
for i, incorrect_idx in enumerate(incorrect[:num_show]):
    plt.subplot(1, num_show, i + 1)
    plt.imshow(X_test[incorrect_idx].astype('uint8'))
    plt.title(f'True: {y_test[incorrect_idx]}, Pred: {y_pred_classes[incorrect_idx]}')
    plt.axis('off')
plt.show()

#6 - Fazer Previsões

##a - Função para carregar novas imagens a partir de um caminho

In [ ]:
def load_new_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalização
    return img_array

##b - Lista de novos caminhos de imagem

In [ ]:
#new_image_paths = ['/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/new_image1.jpg',
 #                  '/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/new_image2.jpg']

 # Usar imagens existentes no diretório Dataset_Arvores
existing_image_paths = [
    '/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/tree01.jpg',
    '/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/tree02.jpg'
]

In [ ]:
# Verificar se os caminhos das imagens existem
for img_path in existing_image_paths:
    if not os.path.exists(img_path):
        print(f'File not found: {img_path}')

In [ ]:
# Se todos os arquivos existirem, carregar e prever
if all(os.path.exists(img_path) for img_path in existing_image_paths):
    new_images = np.vstack([load_new_image(img_path) for img_path in existing_image_paths])
    predictions = model.predict(new_images)
    predicted_classes = np.argmax(predictions, axis=1)

    # Exibir previsões
    for i, img_path in enumerate(existing_image_paths):
        print(f'Image: {img_path} -> Predicted Class: {predicted_classes[i]}')
        img = image.load_img(img_path)
        plt.imshow(img)
        plt.title(f'Predicted Class: {predicted_classes[i]}')
        plt.axis('off')
        plt.show()
else:
    print("One or more image files were not found.")

##c - Carregar e prever novas imagens

In [ ]:
#definir caminho das novas imagens
new_image_paths = [
    '/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/new_tree01.jpg',
    '/content/drive/My Drive/Colab Notebooks/Dataset_Arvores/new_tree02.jpg'
]

In [ ]:
#função para carregar novas imagens
def load_new_image(image_path):
  img = image.load_img(image_path, target_size=(224,224))
  img_array = image.img_to_array(img)
  return np.expand_dims(img_array,axis=0)


In [ ]:
# Função para monitorar o diretório de novas imagens e processar as imagens adicionadas
def monitor_and_predict(model, new_images_dir, interval=60):
    processed_images = set()  # Manter um registro de imagens já processadas

    while True:
        new_image_paths = [os.path.join(new_images_dir, img) for img in os.listdir(new_images_dir) if img.endswith('.jpg') or img.endswith('.png')]

        # Filtrar imagens já processadas
        unprocessed_images = [img for img in new_image_paths if img not in processed_images]

        if unprocessed_images:
            new_images_list = [load_new_image(img_path) for img_path in unprocessed_images]
            new_images = np.vstack(new_images_list)

            # Fazer previsões
            predictions = model.predict(new_images)
            predicted_classes = np.argmax(predictions, axis=1)

            # Exibir as previsões
            for img_path, predicted_class in zip(unprocessed_images, predicted_classes):
                print(f"Image: {img_path}, Predicted Class: {predicted_class}")

            # Atualizar a lista de imagens processadas
            processed_images.update(unprocessed_images)

        # Aguardar antes de verificar novamente
        time.sleep(interval)

In [ ]:
# Função para prever classes de novas imagens
def predict_new_images(model, new_images_dir):
    new_image_paths = [os.path.join(new_images_dir, img) for img in os.listdir(new_images_dir) if img.endswith('.jpg') or img.endswith('.png')]
    new_images_list = [load_new_image(img_path) for img_path in new_image_paths]
    new_images = np.vstack(new_images_list)

In [ ]:
#carregar novas imagens
new_images = np.vstack([load_new_image(img_path) for img_path in new_image_paths])

In [ ]:
new_images = np.vstack([load_new_image(img_path) for img_path in new_image_paths])
predictions = model.predict(new_images)
predicted_classes = np.argmax(predictions, axis=1)


In [ ]:
# Fazer previsões
predictions = model.predict(new_images)
predicted_classes = np.argmax(predictions, axis=1)


In [ ]:
# Exibir as previsões
for img_path, predicted_class in zip(new_image_paths, predicted_classes):
       print(f"Image: {img_path}, Predicted Class: {predicted_class}")

In [ ]:
# Definir o diretório que contém as novas imagens
new_images_dir = '/content/drive/My Drive/Colab Notebooks/Dataset_Arvores'

In [ ]:
#salvar em arquivo csv as predições de queda de árvore
